In [19]:
# Importação de bibliotecas e módulos para resolução de exercícios

from cobra.io import read_sbml_model
from mewpy.simulation import get_simulator
import cobra
import pandas as pd

from mewpy.omics import ExpressionSet
from mewpy.omics import eFlux    
        

In [20]:
# Carregamos o modelo usando 

modelo_base = cobra.io.read_sbml_model('iRC1080_lv3.xml')
modelo_base

Name,iRC1080
Memory address,17f68fd5518
Number of metabolites,1706
Number of reactions,2191
Number of genes,1086
Number of groups,0
Objective expression,1.0*BIOMASS_Chlamy_hetero - 1.0*BIOMASS_Chlamy_hetero_reverse_b1c3b
Compartments,"cytosol, mitochondria, chloroplast, flagellum, peroxisome/glyoxysome, nucleus, golgi apparatus, extracellular space, eyespot, thylakoid"


# Exercise 2

In stress conditions, microalgae  often change  their  metabolism to fight the alterations in the environment. In these scenarios, some carbon can be secreted in the 
form  of  organic acids. Although  some  stress  conditions  can  be  replicated directly  in GSM  models,  others, such as  temperature,  pH,  salinity,  require  the addition of additional  information,  such  as  gene expression  data. The file “expression_data.tsv” contains  the  normalized  gene  expression  profile  of  C.  reinhardii in  two  conditions: control and stress. 

## 2.a. 

Integrate the  expression  data for  both  conditions  using  the  eFLUX  algorithm. 
If your metabolite is H2, use the column “Stress_h2” instead of “Stress”.  The environmental condition  for each group/metabolite is available in Table  1.


In [21]:
'''
Definimos as condições ambientais. 
No caso do nosso grupo as condições são 'microaerobic mixotrophic'

photon -2000
co2 -11.6
o2 -0.01
acetate -10
starch -1.72e-04

'''
modelo = modelo_base.copy()
modelo.objective = 'EX_h2_e'

cond_ambientais = {
        'EX_photonVis_e':(-2000    , 1000),   # condições de luz
        'EX_co2_e'      :(-11.6    , 1000),   # consumo C02
        'EX_o2_e'       :(-0.01    , 1000),   # condições de microaerobiose
        'EX_ac_e'       :(-10      , 1000),   # consumo acetato
        'SK_starch300_h':(-1.72e-4 , 1000),   # consumo amido
                    
          }

modelo

Name,iRC1080
Memory address,17f6e7ed160
Number of metabolites,1706
Number of reactions,2191
Number of genes,1086
Number of groups,0
Objective expression,1.0*EX_h2_e - 1.0*EX_h2_e_reverse_f55e9
Compartments,"cytosol, mitochondria, chloroplast, flagellum, peroxisome/glyoxysome, nucleus, golgi apparatus, extracellular space, eyespot, thylakoid"


In [22]:
'''# POSSIVELMENTE APAGAR POIS SERÁ ALTERADO NO EX 1
# ALTERAMOS AS CONDICOES NO MODELO

for condicao in cond_ambientais.keys():
    mudar = modelo.reactions.get_by_id(condicao)
    mudar.bounds = cond_ambientais[condicao]
    #print(mudar.id, cond_ambientais[condicao])'''

In [23]:
# Carregamento do ficheiro de dados de expressão genética

exp_gen = pd.read_csv('expression_data.tsv',sep='\t',index_col=0)

# Como não nos interessa 'Stress_12' vamos eliminar a coluna

exp_gen.drop('Stress_12h', axis = 1, inplace = True)

In [24]:
exp_gen.columns

Index(['Control_12h', 'Stress_12_h2'], dtype='object')

In [25]:
exp_gen.index.values

array(['Cre01.g000033.v5.5', 'Cre01.g000250.v5.5', 'Cre01.g000300.v5.5',
       ..., 'Cre51.g761547.v5.5', 'Cre51.g761597.v5.5',
       'Cre48.g761197.v5.5'], dtype=object)

In [26]:
# Estabelecemos as condições para a simulação usando MEWpy
simulador = get_simulator(modelo, envcond = cond_ambientais)

# Tratamos os dados para integração

genes      = exp_gen.index.values                     # Isolamos a lista de genes
ccontrolo  = ['Control_12h' ]  # Definimos as nossas condições
cstress    = ['Stress_12_h2']
controlo   = exp_gen[ccontrolo].to_numpy()         # Definimos os dados de expressão para cada uma das condições - controlo
stress     = exp_gen[cstress  ].to_numpy()         # Definimos os dados de expressão para cada uma das condições - stress

# Criamos o módulo de expressão

exp_controlo = ExpressionSet(genes, ccontrolo, controlo)
exp_stress   = ExpressionSet(genes, cstress  , stress  )

In [27]:
# Realizamos a integração dos dados para a condição controlo
# Usamos o algoritmo eFLUX

g_c = eFlux(simulador,exp_controlo)


In [52]:
g_c.find(['H2td']) 

,Flux rate
Reaction ID,
H2td,0.000459


In [29]:
#g_c.fluxes

In [59]:
# Realizamos a integração dos dados para a condição de stress
# Usamos o algoritmo eFLUX

g_s = eFlux(simulador,exp_stress)

In [58]:
g_s.find(['H2td'])

,Flux rate
Reaction ID,
H2td,0.143033


## 2.b

|Controlo|Stress|
|-|-|
|4.59e-4|0.143|

Atendendo aos resultados blablabla a C. reinhcoisas produz mais em situações de stress.

|Resultados 1b|Controlo|Stress|
|-|-|-|
|4.274|0.000459|0.143|




# Exercise 3

## 3.a

In [ ]:
# Fazemos uma cópia do modelo "intacto" , não indica pelo que não vou adaptar às condições microaerobic mixotrophic

modelo = modelo_base.copy()
modelo.objective = 'BIOMASS_Chlamy_mixo'


In [ ]:
# Genes essenciais

genes_ess = cobra.flux_analysis.find_essential_genes(modelo)

In [ ]:
len(genes_ess)

In [ ]:
# Reacções essenciais
reacts_ess = cobra.flux_analysis.find_essential_reactions(modelo)

In [ ]:
len(reacts_ess)

## 3.b

In [ ]:
# Single deletions

# Genes

single_gen_del = cobra.flux_analysis.single_gene_deletion(modelo)


In [ ]:
# Reacoes

single_reac_del = cobra.flux_analysis.single_reaction_deletion(modelo)

In [ ]:
# Double deletions
# Genes

# double_gen_del = cobra.flux_analysis.double_gene_deletion(modelo)

In [ ]:
# Reacoes

# double_reac_del = cobra.flux_analysis.double_reaction_deletion(modelo)

In [ ]:
single_gen_del

In [ ]:
single_reac_del

In [ ]:
double_gen_del

In [ ]:
double_reac_del